# Load data

In [12]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../Data/Datos_CLOFBO-Plata.tsv', sep='\t', encoding='ISO-8859-1')
# Display the first few rows
print(df.head())

df['genus'] = df['scientificName'].str.split(' ').str[0]

   number            type                                           modified  \
0       1  PhysicalObject  Sat Nov 02 2024 00:00:00 GMT+0100 (Central Eur...   
1       2  PhysicalObject  Sat Nov 02 2024 00:00:00 GMT+0100 (Central Eur...   
2       3  PhysicalObject  Sat Nov 02 2024 00:00:00 GMT+0100 (Central Eur...   
3       4  PhysicalObject  Sat Nov 02 2024 00:00:00 GMT+0100 (Central Eur...   
4       5  PhysicalObject  Sat Nov 02 2024 00:00:00 GMT+0100 (Central Eur...   

                               bibliographicCitation     basisOfRecord  \
0  Boulenger, G. A. (1897) Viaggio del Dott. Alfr...  MaterialCitation   
1  Boulenger, G. A. (1897) Viaggio del Dott. Alfr...  MaterialCitation   
2  Boulenger, G. A. (1897) Viaggio del Dott. Alfr...  MaterialCitation   
3  Boulenger, G. A. (1897) Viaggio del Dott. Alfr...  MaterialCitation   
4  Boulenger, G. A. (1897) Viaggio del Dott. Alfr...  MaterialCitation   

  catalogNumber recordNumber       recordedBy individualCount  sex  ...  \

In [8]:
class_levels_list = [level for level in df['class'].unique() if pd.notna(level)]
print(class_levels_list)

# Loop through each class level
for class_level in class_levels_list:
    
    # Filter the dataframe for the current class level
    class_df = df[df['class'] == class_level]
    
    # Do something with class_df here
    # For example, print how many rows belong to this class
    upper_class = class_level.upper()
    print(upper_class)
    spaced_class = ' '.join(upper_class)
    print(spaced_class)



['Actinopterygii', 'Elasmobranchii', 'Dipnoi']
ACTINOPTERYGII
A C T I N O P T E R Y G I I
ELASMOBRANCHII
E L A S M O B R A N C H I I
DIPNOI
D I P N O I


In [20]:
from docx import Document
from docx.shared import Pt  # for font size in points
from docx.oxml.ns import qn  # for setting font name properly
from docx.shared import Inches, Mm

# Create a new Document
doc = Document()

# Set the document layout
section = doc.sections[0]

# Set to A4 size (210mm x 297mm)
section.page_height = Mm(297)
section.page_width = Mm(210)

section.top_margin = Inches(0.98)
section.bottom_margin = Inches(0.79)
section.left_margin = Inches(0.98)
section.right_margin = Inches(0.69)

# Title

# Title line
title = doc.add_paragraph()

# First run: small text
run1 = title.add_run("Check List of the Freshwater Fishes from the La Plata basin in Bolivia (CLOFFBO-Plata).")
run1.font.name = 'Arial'
run1.font.size = Pt(12)
run1.font.bold = True

class_levels_list = [level for level in df['class'].unique() if pd.notna(level)]
print(class_levels_list)

# Loop through each class level
for class_level in class_levels_list:
    
    # Filter the dataframe for the current class level
    class_df = df[df['class'] == class_level]
    
    # Do something with class_df here
    # For example, print how many rows belong to this class
    upper_class = class_level.upper()
    print(upper_class)
    spaced_class = ' '.join(upper_class)
    print(spaced_class)
    
    # Class line
    class_ = doc.add_paragraph()

    # First run: small text
    run1 = class_.add_run("class	")
    run1.font.name = 'Arial'
    run1.font.size = Pt(10)

    # Second run: larger text
    run2 = class_.add_run(spaced_class)
    run2.font.name = 'Arial'
    run2.font.size = Pt(12)
    
    # Loop through each order level
    order_levels_list = [level for level in class_df['order'].unique() if pd.notna(level)]
    print(order_levels_list)
    
    for order_level in order_levels_list:
    
        # Filter the dataframe for the current class level
        order_df = class_df[class_df['order'] == order_level]
    
        # Order line
        order = doc.add_paragraph()

        # First run: small text
        run1 = order.add_run("order		")
        run1.font.name = 'Arial'
        run1.font.size = Pt(10)

        # Second run: larger text
        run2 = order.add_run(order_level)
        run2.font.name = 'Arial'
        run2.font.size = Pt(12)
        run2.font.bold = True
        
        # Loop through each family level
        family_levels_list = [level for level in order_df['family'].unique() if pd.notna(level)]
        print(family_levels_list)
    
        for family_level in family_levels_list:
    
            # Filter the dataframe for the current class level
            family_df = order_df[order_df['family'] == family_level]        
        
            # Family line
            order = doc.add_paragraph()

            # First run: small text
            run1 = order.add_run("family			")
            run1.font.name = 'Arial'
            run1.font.size = Pt(10)

            # Second run: larger text
            run2 = order.add_run(family_level)
            run2.font.name = 'Arial'
            run2.font.size = Pt(12)
            run2.font.bold = True
            
            # Loop through each family level
            genus_levels_list = [level for level in family_df['genus'].unique() if pd.notna(level)]
            print(genus_levels_list)
    
            for genus_level in genus_levels_list:
    
                # Filter the dataframe for the current class level
                genus_df = family_df[family_df['genus'] == genus_level] 
            
                # Genus line
                genus = doc.add_paragraph()

                # First run: small text
                run1 = genus.add_run(genus_level)
                run1.font.name = 'Arial'
                run1.font.size = Pt(10)
                run1.font.bold = True
                run1.font.italic = True
                
                # Loop through each genus level
                species_levels_list = [level for level in genus_df['specificEpithet'].unique() if pd.notna(level)]
                print(species_levels_list)
    
                for species_level in species_levels_list:
    
                    # Filter the dataframe for the current class level
                    species_df = genus_df[genus_df['specificEpithet'] == species_level] 
                    author = species_df['scientificNameAuthorship'].iloc[0]
                
                    # Species line
                    species = doc.add_paragraph()

                    run = species.add_run("	" + genus_level)
                    run.font.name = 'Arial'
                    run.font.size = Pt(10)
                    run.font.bold = True
                    run.font.italic = True
                    
                    run = species.add_run(" " + species_level)
                    run.font.name = 'Arial'
                    run.font.size = Pt(10)
                    run.font.bold = True
                    run.font.italic = True
                    
                    run = species.add_run(" " + author)
                    run.font.name = 'Arial'
                    run.font.size = Pt(10)
                    run.font.bold = True
                    run.font.italic = True
                
                empty_line = doc.add_paragraph()
                run = empty_line.add_run("")
                run.font.name = 'Arial'
                run.font.size = Pt(10)

# Save the document
doc.save('../Data/CLOFBO-Plata.docx')

['Actinopterygii', 'Elasmobranchii', 'Dipnoi']
ACTINOPTERYGII
A C T I N O P T E R Y G I I
['Siluriformes', 'Characiformes', 'Cyprinodontiformes', 'Synbranchiformes', 'Cichliformes', 'Gymnotiformes', 'Beloniformes', 'Clupeiformes', 'Acanthuriformes', 'Pleuronectiformes']
['Pimelodidae', 'Heptapteridae', 'Loricariidae', 'Trichomycteridae', 'Callichthyidae', 'Doradidae', 'Auchenipteridae', 'Aspredinidae', 'Pseudopimelodidae', 'Cetopsidae']
['Pimelodus', 'Pseudoplatystoma', 'Megalonema', 'Sorubim', 'Luciopimelodus', 'Zungaro', 'Pinirampus']


KeyError: 'specificEpithet'

In [ ]:
from docx import Document
from docx.shared import Pt  # for font size in points
from docx.oxml.ns import qn  # for setting font name properly
from docx.shared import Inches, Mm

# Create a new Document
doc = Document()

# Set the document layout
section = doc.sections[0]

# Set to A4 size (210mm x 297mm)
section.page_height = Mm(297)
section.page_width = Mm(210)

section.top_margin = Inches(0.98)
section.bottom_margin = Inches(0.79)
section.left_margin = Inches(0.98)
section.right_margin = Inches(0.69)

# Title

# Title line
title = doc.add_paragraph()

# First run: small text
run1 = title.add_run("Check List of the Freshwater Fishes from the La Plata basin in Bolivia (CLOFFBO-Plata).")
run1.font.name = 'Arial'
run1.font.size = Pt(12)
run1.font.bold = True

# Class line
class_ = doc.add_paragraph()

# First run: small text
run1 = class_.add_run("class	")
run1.font.name = 'Arial'
run1.font.size = Pt(10)

# Second run: larger text
run2 = class_.add_run("E L A S M O B R A N C H I I")
run2.font.name = 'Arial'
run2.font.size = Pt(12)

# Order line
order = doc.add_paragraph()

# First run: small text
run1 = order.add_run("order		")
run1.font.name = 'Arial'
run1.font.size = Pt(10)

# Second run: larger text
run2 = order.add_run("RAJIFORMES")
run2.font.name = 'Arial'
run2.font.size = Pt(12)
run2.font.bold = True

# Family line
order = doc.add_paragraph()

# First run: small text
run1 = order.add_run("family			")
run1.font.name = 'Arial'
run1.font.size = Pt(10)

# Second run: larger text
run2 = order.add_run("Potamotrygonidae")
run2.font.name = 'Arial'
run2.font.size = Pt(12)
run2.font.bold = True

# Genus line
genus = doc.add_paragraph()

# First run: small text
run1 = genus.add_run("Potamotrygon")
run1.font.name = 'Arial'
run1.font.size = Pt(10)
run1.font.bold = True
run1.font.italic = True

# Species line
species = doc.add_paragraph()

run = species.add_run("	P. falkneri")
run.font.name = 'Arial'
run.font.size = Pt(10)
run.font.bold = True
run.font.italic = True

run = species.add_run("   Castex & Maciel, 1963")
run.font.name = 'Arial'
run.font.size = Pt(10)
run.add_break()

run = species.add_run("			synonyms")
run.font.name = 'Arial'
run.font.size = Pt(10)

run = species.add_run("	P. castexi")
run.font.name = 'Arial'
run.font.size = Pt(10)
run.font.italic = True

run = species.add_run(" Castello & Yagolkowski, 1969")
run.font.name = 'Arial'
run.font.size = Pt(10)
run.add_break()

run = species.add_run("			record")
run.font.name = 'Arial'
run.font.size = Pt(8)

run = species.add_run("		lista de Karina sub")
run.font.name = 'Arial'
run.font.size = Pt(8)

run = species.add_run(" P. castexi")
run.font.name = 'Arial'
run.font.size = Pt(8)
run.font.italic = True

# Save the document
doc.save('../Data/CLOFBO-Plata.docx')